# 통합 식품영양성분DB 정리
### - [클리닝](#1.-클리닝)
### - [타입 변경(object -> float)](#2.-타입-변경)
### - [데이터 단위 통일(영양소 g으로 통일)](#3.-영양소-별-데이터-단위-통일)
### - [각 식품의 영양소 최대 값 행 추가](#4.-)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re


In [2]:
food_nutrients = pd.read_excel('통합 식품영양성분DB_20211106.xlsx', skiprows=[0,1,2])
food_nutrients.head()

,NO,SAMPLE_ID,식품코드,DB군,상용제품,식품명,연도,지역 / 제조사,채취시기,식품대분류,...,냉산가용성물질(㎎),총 불포화지방산(g),식염상당량(g),회분(g),폐기율(%),가식부(%),산가용성물질(%),카페인(㎎),성분표출처,발행기관
0,1,D000006-94-AVG,D000006,음식,품목대표,꿩불고기,2019,충주,평균,구이류,...,-,-,-,5.8,-,-,-,-,식약처('16) 제4권,식품의약품안전처
1,2,D000007-ZZ-AVG,D000007,음식,품목대표,닭갈비,2019,전국(대표),평균,구이류,...,-,-,-,7,-,-,-,-,식약처('16) 제4권,식품의약품안전처
2,3,D000008-66-AVG,D000008,음식,품목대표,닭갈비,2019,춘천,평균,구이류,...,-,-,-,4.5,-,-,-,-,식약처('16) 제4권,식품의약품안전처
3,4,D000009-ZZ-AVG,D000009,음식,품목대표,닭꼬치,2019,전국(대표),평균,구이류,...,-,-,-,1.2176954799497,-,-,-,-,식약처('12) 제1권,식품의약품안전처
4,5,D000010-ZZ-AVG,D000010,음식,품목대표,더덕구이,2019,전국(대표),평균,구이류,...,-,-,-,2.5,-,-,-,-,식약처('17) 제5권,식품의약품안전처


In [3]:
pd.options.display.max_columns = 241
pd.set_option('display.max.colwidth', 50)

food_nutrients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58597 entries, 0 to 58596
Columns: 241 entries, NO to 발행기관
dtypes: float64(1), int64(2), object(238)
memory usage: 107.7+ MB


## 1. 클리닝 

#### 필요없는 colum 제거 및 float로 변환하기 위해 데이터 정제

In [4]:
food_nutrients = food_nutrients.drop(['SAMPLE_ID', '식품코드', '연도', '지역 / 제조사',\
                                      '채취시기', '폐기율(%)', '성분표출처', '발행기관'], axis = 1)

In [5]:
food_nutrients.head()

,NO,DB군,상용제품,식품명,식품대분류,식품상세분류,1회제공량,내용량_단위,총내용량(g),총내용량(mL),에너지(㎉),에너지(kj),수분(g),수분(%),단백질(g),지방(g),지질-가식부 100g당,탄수화물(g),총당류(g),자당(g),포도당(g),과당(g),유당(g),맥아당(g),갈락토오스(g),당알콜(g),에리스리톨(g),총 식이섬유(g),총 식이섬유(mg),총 식이섬유(%),수용성 식이섬유(g),불용성 식이섬유(g),셀룰로오스(%),리그닌(%),칼슘(㎎),철(㎎),철(㎍),마그네슘(㎎),인(㎎),칼륨(g),칼륨(㎎),나트륨(㎎),아연(㎎),구리(㎎),구리(㎍),망간(㎎),망간(㎍),셀레늄(㎍),몰리브덴(㎍),요오드(㎍),염소(㎎),비타민 A(㎍),비타민 A(㎍ RE),레티놀(㎍),베타카로틴(㎍),레티놀A효능,비타민 D(D2+D3)(㎍),비타민 D2(㎍),비타민 D3(㎍),비타민 D1(㎍),비타민 E(㎎),비타민 E(㎎ α-TE),알파 토코페롤(㎎),베타 토코페롤(㎎),감마 토코페롤(㎎),델타 토코페롤(㎎),알파 토코트리에놀(㎎),베타 토코트리에놀(㎎),감마 토코트리에놀(㎎),델타 토코트리에놀(㎎),토코페롤(㎎),토코트리에놀(㎎),비타민 K(㎎),비타민 K(㎍),비타민 K1(㎍),비타민 K2(㎍),비타민 B1(㎎),비타민 B1(㎍),비타민 B2(㎎),비타민 B2(㎍),나이아신(㎎),나이아신(㎎ NE),나이아신(NE)(㎎),나이아신(NE)(㎎ NE),니코틴산 (㎎),니코틴아마이드(㎎),판토텐산(㎎),판토텐산(㎍),비타민 B6(㎎),비타민 B6(㎍),피리독신(㎎),비오틴(㎍),엽산(DFE)(㎍),엽산 - 천연 엽산(㎍),엽산 - 합성 엽산(㎍),비타민 B12(㎎),비타민 B12(㎍),비타민 C(g),비타민 C(㎎),콜린(g),콜린(㎎),총 아미노산(g),총 아미노산(㎎),필수 아미노산(㎎),이소류신(㎎),류신(㎎),라이신(㎎),메티오닌(㎎),페닐알라닌(㎎),트레오닌(㎎),트립토판(㎎),발린(㎎),히스티딘(㎎),아르기닌(㎎),비필수 아미노산(㎎),티로신(㎎),시스테인(㎎),알라닌(㎎),아스파르트산(㎎),글루탐산(㎎),글리신(㎎),프롤린(㎎),세린(㎎),타우린(㎎),글리신 베타인 (㎎),호마린(㎎),트리고넬린(㎎),리보핵산(㎎),데옥시리보핵산(㎎),콜레스테롤(g),콜레스테롤(㎎),총 지방산(g),총 필수 지방산(g),총 포화 지방산(g),총 포화 지방산(%),부티르산(4:0)(g),부티르산(4:0)(㎎),카프로산(6:0)(g),카프로산(6:0)(㎎),카프릴산(8:0)(g),카프릴산(8:0)(㎎),카프르산(10:0)(g),카프르산(10:0)(㎎),라우르산(12:0)(g),라우르산(12:0)(㎎),라우르산(12:0)(%),트라이데칸산(13:0)(㎎),미리스트산(14:0)(g),미리스트산(14:0)(㎎),미리스트산(14:0)(%),펜타데칸산(15:0)(㎎),팔미트산(16:0)(g),팔미트산(16:0)(㎎),팔미트산(16:0)(%),헵타데칸산(17:0)(㎎),스테아르산(18:0)(g),스테아르산(18:0)(㎎),스테아르산(18:0)(%),아라키드산(20:0)(g),아라키드산(20:0)(㎎),아라키드산(20:0)(%),헨에이코산산(21:0)(㎎),베헨산(22:0)(㎎),트리코산산(23:0)(㎎),리그노세르산(24:0)(㎎),총 단일 불포화지방산(g),총 단일 불포화지방산(%),미리스톨레산(14:1)(g),미리스톨레산(14:1)(㎎),미리스톨레산(14:1)(%),팔미톨레산(16:1)(g),팔미톨레산(16:1)(㎎),팔미톨레산(16:1)(%),헵타데센산(17:1)(㎎),올레산(18:1(n-9))(g),올레산(18:1(n-9))(㎎),올레산(18:1(n-9))(%),박센산(18:1(n-7))(g),박센산(18:1(n-7))(㎎),가돌레산(20:1)(g),가돌레산(20:1)(㎎),가돌레산(20:1)(%),에루크산(22:1)(㎎),에루크산(22:1)(%),네르본산(24:1)(㎎),총 다중 불포화지방산(g),총 다중 불포화지방산(%),리놀레산(18:2(n-6)c)(g),리놀레산(18:2(n-6)c)(㎎),리놀레산(18:2(n-6)c)(%),알파 리놀렌산(18:3(n-3))(g),알파 리놀렌산(18:3(n-3))(㎎),리놀렌산(18:3(n-3)(%),감마 리놀렌산(18:3(n-6))(g),감마 리놀렌산(18:3(n-6))(㎎),스테아리돈산(18:4)(%),에이코사디에노산(20:2(n-6))(g),에이코사디에노산(20:2(n-6))(㎎),에이코사트리에노산(20:3(n-3))(㎎),에이코사트리에노산(20:3(n-6))(g),에이코사트리에노산(20:3(n-6))(㎎),아라키돈산(20:4(n-6))(g),아라키돈산(20:4(n-6))(㎎),아라키돈산(20:4(n-6))(%),에이코사테트라에노산(20:4(n-3))(㎎),에이코사펜타에노산(20:5(n-3))(g),에이코사펜타에노산(20:5(n-3))(㎎),에이코사펜타에노산(20:5(n-3))(%),도코사디에노산(22:2)(㎎),도코사펜타에노산(22:5(n-3))(g),도코사펜타에노산(22:5(n-3))(㎎),도코사펜타에노산(22:5(n-3))(%),도코사헥사에노산(22:6(n-3))(g),도코사헥사에노산(22:6(n-3))(㎎),도코사헥사에노산(22:6(n-3))(%),EPA와 DHA의 합(㎎),오메가 3 지방산(g),오메가 6 지방산(g),트랜스 지방산(g),트랜스 올레산(18:1(n-9)t)(g),트랜스 올레산(18:1(n-9)t)(㎎),트랜스 리놀레산 (18:2t)(g),트랜스 리놀레산 (18:2t)(㎎),트랜스 리놀렌산(18:3t)(g),트랜스 리놀렌산(18:3t)(㎎),트랜스 리놀렌산(18:3t)(%),냉산가용성물질(㎎),총 불포화지방산(g),식염상당량(g),회분(g),가식부(%),산가용성물질(%),카페인(㎎)
0,1,음식,품목대표,꿩불고기,구이류,육류구이,500.0,g,-,-,368.8,-,412.6,-,33.5,8.5,-,39.7,16.9,7.2,2.8,2.8,0.7,3.5,-,-,-,9.8,-,-,-,-,-,-,105.61,-,4,85.39,458.05,-,1243.12,1264.31,3.99,0.32,-,0.68,-,47.55,-,-,-,-,-,0,1424.58,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,8.73,0.06,-,-,-,-,0,-,0.33,-,3.61,-,-,-,-,-,-,-,-,-,-,-,63.13,-,-,-,0,-,2.99,-,-,-,-,-,1284.37582123574,2328.59892766839,2484.75047814224,645.190070073169,1274.92730261488,1410.84127771718,-,1367.68831341948,848.539312406619,2205.5847774439,-,919.805669181432,276.498720836072,1642.38119022329,2781.77304833044,4784.60634708081,1419.92479570739,1092.29548260871,1264.95795056827,-,-,-,-,-,-,-,106.18,-,-,1.9,-,0,-,0,-,0,-,0,-,0.00711483471961893,-,-,-,0.0317835536096363,-,-,-,1.28384114493856,-,-,-,0.527395267901326,-,-,0.0290123491481805,-,-,-,-,-,-,-,-,0,-,-,0.0813516573054785,-,-,-,2.30772104748742,-,-,0.113462530850897,-,0.0279762322988018,-,-,-,-,-,-,-,3.23601791813763,-,-,0.656722581376586,-,-,0.018708284652282,-,-,0.00664887007716417,-,-,0,-,0.142840481460645,-,-,-,0,-,-,-,0,-,-,0.0234269823295255,-,-,-,-,-,0.1,0.018116327971628,-,0.0534160839712029,-,0,-,-,-,-,-,5.8,-,-,-
1,2,음식,품목대표,닭갈비,구이류,육류구이,400.0,g,-,-,595.61,-,276.4,-,45.9,25.8,

In [6]:
food_nutrients_copy = food_nutrients.copy()
food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'] = food_nutrients.loc[: ,'단백질(g)':'카페인(㎎)'].replace(['-', 'tr', 'Tr'], 0)

In [7]:
food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'] = \
        food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'].replace(re.compile(r'미만'), 0)
food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'] = \
        food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'].replace(re.compile(r'이하'), 0)
food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'] = \
        food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'].replace(re.compile(r'<'), 0)

food_nutrients_copy.head()

,NO,DB군,상용제품,식품명,식품대분류,식품상세분류,1회제공량,내용량_단위,총내용량(g),총내용량(mL),에너지(㎉),에너지(kj),수분(g),수분(%),단백질(g),지방(g),지질-가식부 100g당,탄수화물(g),총당류(g),자당(g),포도당(g),과당(g),유당(g),맥아당(g),갈락토오스(g),당알콜(g),에리스리톨(g),총 식이섬유(g),총 식이섬유(mg),총 식이섬유(%),수용성 식이섬유(g),불용성 식이섬유(g),셀룰로오스(%),리그닌(%),칼슘(㎎),철(㎎),철(㎍),마그네슘(㎎),인(㎎),칼륨(g),칼륨(㎎),나트륨(㎎),아연(㎎),구리(㎎),구리(㎍),망간(㎎),망간(㎍),셀레늄(㎍),몰리브덴(㎍),요오드(㎍),염소(㎎),비타민 A(㎍),비타민 A(㎍ RE),레티놀(㎍),베타카로틴(㎍),레티놀A효능,비타민 D(D2+D3)(㎍),비타민 D2(㎍),비타민 D3(㎍),비타민 D1(㎍),비타민 E(㎎),비타민 E(㎎ α-TE),알파 토코페롤(㎎),베타 토코페롤(㎎),감마 토코페롤(㎎),델타 토코페롤(㎎),알파 토코트리에놀(㎎),베타 토코트리에놀(㎎),감마 토코트리에놀(㎎),델타 토코트리에놀(㎎),토코페롤(㎎),토코트리에놀(㎎),비타민 K(㎎),비타민 K(㎍),비타민 K1(㎍),비타민 K2(㎍),비타민 B1(㎎),비타민 B1(㎍),비타민 B2(㎎),비타민 B2(㎍),나이아신(㎎),나이아신(㎎ NE),나이아신(NE)(㎎),나이아신(NE)(㎎ NE),니코틴산 (㎎),니코틴아마이드(㎎),판토텐산(㎎),판토텐산(㎍),비타민 B6(㎎),비타민 B6(㎍),피리독신(㎎),비오틴(㎍),엽산(DFE)(㎍),엽산 - 천연 엽산(㎍),엽산 - 합성 엽산(㎍),비타민 B12(㎎),비타민 B12(㎍),비타민 C(g),비타민 C(㎎),콜린(g),콜린(㎎),총 아미노산(g),총 아미노산(㎎),필수 아미노산(㎎),이소류신(㎎),류신(㎎),라이신(㎎),메티오닌(㎎),페닐알라닌(㎎),트레오닌(㎎),트립토판(㎎),발린(㎎),히스티딘(㎎),아르기닌(㎎),비필수 아미노산(㎎),티로신(㎎),시스테인(㎎),알라닌(㎎),아스파르트산(㎎),글루탐산(㎎),글리신(㎎),프롤린(㎎),세린(㎎),타우린(㎎),글리신 베타인 (㎎),호마린(㎎),트리고넬린(㎎),리보핵산(㎎),데옥시리보핵산(㎎),콜레스테롤(g),콜레스테롤(㎎),총 지방산(g),총 필수 지방산(g),총 포화 지방산(g),총 포화 지방산(%),부티르산(4:0)(g),부티르산(4:0)(㎎),카프로산(6:0)(g),카프로산(6:0)(㎎),카프릴산(8:0)(g),카프릴산(8:0)(㎎),카프르산(10:0)(g),카프르산(10:0)(㎎),라우르산(12:0)(g),라우르산(12:0)(㎎),라우르산(12:0)(%),트라이데칸산(13:0)(㎎),미리스트산(14:0)(g),미리스트산(14:0)(㎎),미리스트산(14:0)(%),펜타데칸산(15:0)(㎎),팔미트산(16:0)(g),팔미트산(16:0)(㎎),팔미트산(16:0)(%),헵타데칸산(17:0)(㎎),스테아르산(18:0)(g),스테아르산(18:0)(㎎),스테아르산(18:0)(%),아라키드산(20:0)(g),아라키드산(20:0)(㎎),아라키드산(20:0)(%),헨에이코산산(21:0)(㎎),베헨산(22:0)(㎎),트리코산산(23:0)(㎎),리그노세르산(24:0)(㎎),총 단일 불포화지방산(g),총 단일 불포화지방산(%),미리스톨레산(14:1)(g),미리스톨레산(14:1)(㎎),미리스톨레산(14:1)(%),팔미톨레산(16:1)(g),팔미톨레산(16:1)(㎎),팔미톨레산(16:1)(%),헵타데센산(17:1)(㎎),올레산(18:1(n-9))(g),올레산(18:1(n-9))(㎎),올레산(18:1(n-9))(%),박센산(18:1(n-7))(g),박센산(18:1(n-7))(㎎),가돌레산(20:1)(g),가돌레산(20:1)(㎎),가돌레산(20:1)(%),에루크산(22:1)(㎎),에루크산(22:1)(%),네르본산(24:1)(㎎),총 다중 불포화지방산(g),총 다중 불포화지방산(%),리놀레산(18:2(n-6)c)(g),리놀레산(18:2(n-6)c)(㎎),리놀레산(18:2(n-6)c)(%),알파 리놀렌산(18:3(n-3))(g),알파 리놀렌산(18:3(n-3))(㎎),리놀렌산(18:3(n-3)(%),감마 리놀렌산(18:3(n-6))(g),감마 리놀렌산(18:3(n-6))(㎎),스테아리돈산(18:4)(%),에이코사디에노산(20:2(n-6))(g),에이코사디에노산(20:2(n-6))(㎎),에이코사트리에노산(20:3(n-3))(㎎),에이코사트리에노산(20:3(n-6))(g),에이코사트리에노산(20:3(n-6))(㎎),아라키돈산(20:4(n-6))(g),아라키돈산(20:4(n-6))(㎎),아라키돈산(20:4(n-6))(%),에이코사테트라에노산(20:4(n-3))(㎎),에이코사펜타에노산(20:5(n-3))(g),에이코사펜타에노산(20:5(n-3))(㎎),에이코사펜타에노산(20:5(n-3))(%),도코사디에노산(22:2)(㎎),도코사펜타에노산(22:5(n-3))(g),도코사펜타에노산(22:5(n-3))(㎎),도코사펜타에노산(22:5(n-3))(%),도코사헥사에노산(22:6(n-3))(g),도코사헥사에노산(22:6(n-3))(㎎),도코사헥사에노산(22:6(n-3))(%),EPA와 DHA의 합(㎎),오메가 3 지방산(g),오메가 6 지방산(g),트랜스 지방산(g),트랜스 올레산(18:1(n-9)t)(g),트랜스 올레산(18:1(n-9)t)(㎎),트랜스 리놀레산 (18:2t)(g),트랜스 리놀레산 (18:2t)(㎎),트랜스 리놀렌산(18:3t)(g),트랜스 리놀렌산(18:3t)(㎎),트랜스 리놀렌산(18:3t)(%),냉산가용성물질(㎎),총 불포화지방산(g),식염상당량(g),회분(g),가식부(%),산가용성물질(%),카페인(㎎)
0,1,음식,품목대표,꿩불고기,구이류,육류구이,500.0,g,-,-,368.8,-,412.6,-,33.5,8.5,0,39.7,16.9,7.2,2.8,2.8,0.7,3.5,0,0,0,9.8,0,0,0,0,0,0,105.61,0,4,85.39,458.05,0,1243.12,1264.31,3.99,0.32,0,0.68,0,47.55,0,0,0,0,0,0,1424.58,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.73,0.06,0,0,0,0,0,0,0.33,0,3.61,0,0,0,0,0,0,0,0,0,0,0,63.13,0,0,0,0,0,2.99,0,0,0,0,0,1284.37582123574,2328.59892766839,2484.75047814224,645.190070073169,1274.92730261488,1410.84127771718,0,1367.68831341948,848.539312406619,2205.5847774439,0,919.805669181432,276.498720836072,1642.38119022329,2781.77304833044,4784.60634708081,1419.92479570739,1092.29548260871,1264.95795056827,0,0,0,0,0,0,0,106.18,0,0,1.9,0,0,0,0,0,0,0,0,0,0.00711483471961893,0,0,0,0.0317835536096363,0,0,0,1.28384114493856,0,0,0,0.527395267901326,0,0,0.0290123491481805,0,0,0,0,0,0,0,0,0,0,0,0.0813516573054785,0,0,0,2.30772104748742,0,0,0.113462530850897,0,0.0279762322988018,0,0,0,0,0,0,0,3.23601791813763,0,0,0.656722581376586,0,0,0.018708284652282,0,0,0.00664887007716417,0,0,0,0,0.142840481460645,0,0,0,0,0,0,0,0,0,0,0.0234269823295255,0,0,0,0,0,0.1,0.018116327971628,0,0.0534160839712029,0,0,0,0,0,0,0,5.8,0,0,0
1,2,음식,품목대표,닭갈비,구이류,육류구이,400.0,g,-,-,595.61,-,276.4,-,45.9,25.8,

## 2. 타입 변경

In [8]:
food_nutrients_copy = food_nutrients_copy.astype({'1회제공량':'float'})
food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'] = food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'].astype(float)



## 3. 영양소 별 데이터 단위 통일

In [9]:
food_nutrients_copy2 = food_nutrients_copy.copy()

In [10]:
gTomg = 0.001

for col in food_nutrients_copy2.loc[: ,'단백질(g)':'카페인(㎎)']:
    if '㎎' in col:
        food_nutrients_copy2[col] = food_nutrients_copy2[col] * gTomg
for col in food_nutrients_copy2.loc[: ,'단백질(g)':'카페인(㎎)']:
    if '㎍' in col:
        food_nutrients_copy2[col] = food_nutrients_copy2[col] * gTomg * gTomg


In [11]:
food_nutrients_copy2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58597 entries, 0 to 58596
Columns: 233 entries, NO to 카페인(㎎)
dtypes: float64(220), int64(1), object(12)
memory usage: 104.2+ MB


##### 영향소별 비율을 구하기 위해 데이터를 살펴본 결과 데이터가 복잡.?
ex)아라키돈산, 에이코사펜타에노산 같은 영양소 데이터의 경우 g, mg, %로 나뉘어져 있으며 식품에 따라 표기된 위치가 다르다 다행히 g에도 적혀있고 mg에도 적히는 등 중복으로 된 것은 없어서 통합하면 될 것 같다. % 에 표기된 경우 연산 없이 바로 %값을 사용 -> %값이 있는 데이터의 경우 연산을 진행할 때 제외하고 한다....


In [12]:
def make_nutrientt_list(dataframe, _list, giho):
    for col in dataframe:
        if giho in col:
             _list.append(col)

def make_nutrientt_list_delete_giho(dataframe, _list, giho):
    for col in dataframe:
        if giho in col:
            parentheses = col.find('giho')
            _list.append(col[:parentheses - 2 ])

#
nutrient_persent_list = []     #퍼센트로 표기된 영양소 리스트
nutrient_gram_list = []        #그람으로 표기된 영양소 리스트
nutrient_milligram_list = []   #밀리그람으로 표기된 영양소 리스트
nutrient_microgram_list = []   #마이크로 그람으로 표기된 영양소 리스트

#기호 제거 리스트
_nutrient_persent_list = []
_nutrient_gram_list = []
_nutrient_milligram_list = []
_nutrient_microgram_list = []

fn = food_nutrients_copy2.loc[: ,'단백질(g)':'카페인(㎎)']
make_nutrientt_list(fn, nutrient_persent_list, '%')
make_nutrientt_list(fn, nutrient_gram_list, 'g')
make_nutrientt_list(fn, nutrient_milligram_list, '㎎')
make_nutrientt_list(fn, nutrient_microgram_list, '㎍')

make_nutrientt_list_delete_giho(fn, _nutrient_persent_list, '%')
make_nutrientt_list_delete_giho(fn, _nutrient_gram_list, 'g')
make_nutrientt_list_delete_giho(fn, _nutrient_milligram_list, '㎎')
make_nutrientt_list_delete_giho(fn, _nutrient_microgram_list, '㎍')


['총 식이섬유',
 '셀룰로오스',
 '리그닌',
 '총 포화 지방산',
 '라우르산(12:0)',
 '미리스트산(14:0)',
 '팔미트산(16:0)',
 '스테아르산(18:0)',
 '아라키드산(20:0)',
 '총 단일 불포화지방산',
 '미리스톨레산(14:1)',
 '팔미톨레산(16:1)',
 '올레산(18:1(n-9))',
 '가돌레산(20:1)',
 '에루크산(22:1)',
 '총 다중 불포화지방산',
 '리놀레산(18:2(n-6)c)',
 '리놀렌산(18:3(n-3)',
 '스테아리돈산(18:4)',
 '아라키돈산(20:4(n-6))',
 '에이코사펜타에노산(20:5(n-3))',
 '도코사펜타에노산(22:5(n-3))',
 '도코사헥사에노산(22:6(n-3))',
 '트랜스 리놀렌산(18:3t)',
 '가식부',
 '산가용성물질']

In [13]:
(food_nutrients_copy2[nutrient_persent_list]>0.0).sum()

integrated_G_MG = []
integrated_MG_UG = []

for nutrient in _nutrient_milligram_list:
    if nutrient in _nutrient_gram_list:
        integrated_G_MG.append(nutrient)
    if nutrient in _nutrient_microgram_list:
        integrated_MG_UG.append(nutrient)

have_percent = []
for nutrient in _nutrient_persent_list:
    if nutrient in _nutrient_gram_list:
        have_percent.append(nutrient + 'g')
    if nutrient in _nutrient_microgram_list:
        have_percent.append(nutrient + '㎎')

# print(integrated_G_MG, integrated_MG_UG, have_percent)
food_nutrients_copy2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58597 entries, 0 to 58596
Columns: 233 entries, NO to 카페인(㎎)
dtypes: float64(220), int64(1), object(12)
memory usage: 104.2+ MB


#### -단위가 나뉘어서 작성된 영양소 통합 작업-
##### mg의 데이터들은 g으로 옮기고 ug의 데이터들은 mg으로 옮기고 옮긴 colum 삭제

In [14]:
# mg의 데이터들을 g으로 옮겨준다.
for nutrien in integrated_G_MG:
    food_nutrients_copy2[nutrien+'(g)'] = food_nutrients_copy2[nutrien+'(g)'].add(food_nutrients_copy2[nutrien+'(㎎)'])
# ug의 데이터들을 mg으로 옮겨준다.
for nutrien in integrated_MG_UG:
    food_nutrients_copy2[nutrien+'(㎎)'] = food_nutrients_copy2[nutrien+'(㎎)'].add(food_nutrients_copy2[nutrien+'(㎍)'])

In [15]:
food_nutrients_copy2.head()

,NO,DB군,상용제품,식품명,식품대분류,식품상세분류,1회제공량,내용량_단위,총내용량(g),총내용량(mL),에너지(㎉),에너지(kj),수분(g),수분(%),단백질(g),지방(g),지질-가식부 100g당,탄수화물(g),총당류(g),자당(g),포도당(g),과당(g),유당(g),맥아당(g),갈락토오스(g),당알콜(g),에리스리톨(g),총 식이섬유(g),총 식이섬유(mg),총 식이섬유(%),수용성 식이섬유(g),불용성 식이섬유(g),셀룰로오스(%),리그닌(%),칼슘(㎎),철(㎎),철(㎍),마그네슘(㎎),인(㎎),칼륨(g),칼륨(㎎),나트륨(㎎),아연(㎎),구리(㎎),구리(㎍),망간(㎎),망간(㎍),셀레늄(㎍),몰리브덴(㎍),요오드(㎍),염소(㎎),비타민 A(㎍),비타민 A(㎍ RE),레티놀(㎍),베타카로틴(㎍),레티놀A효능,비타민 D(D2+D3)(㎍),비타민 D2(㎍),비타민 D3(㎍),비타민 D1(㎍),비타민 E(㎎),비타민 E(㎎ α-TE),알파 토코페롤(㎎),베타 토코페롤(㎎),감마 토코페롤(㎎),델타 토코페롤(㎎),알파 토코트리에놀(㎎),베타 토코트리에놀(㎎),감마 토코트리에놀(㎎),델타 토코트리에놀(㎎),토코페롤(㎎),토코트리에놀(㎎),비타민 K(㎎),비타민 K(㎍),비타민 K1(㎍),비타민 K2(㎍),비타민 B1(㎎),비타민 B1(㎍),비타민 B2(㎎),비타민 B2(㎍),나이아신(㎎),나이아신(㎎ NE),나이아신(NE)(㎎),나이아신(NE)(㎎ NE),니코틴산 (㎎),니코틴아마이드(㎎),판토텐산(㎎),판토텐산(㎍),비타민 B6(㎎),비타민 B6(㎍),피리독신(㎎),비오틴(㎍),엽산(DFE)(㎍),엽산 - 천연 엽산(㎍),엽산 - 합성 엽산(㎍),비타민 B12(㎎),비타민 B12(㎍),비타민 C(g),비타민 C(㎎),콜린(g),콜린(㎎),총 아미노산(g),총 아미노산(㎎),필수 아미노산(㎎),이소류신(㎎),류신(㎎),라이신(㎎),메티오닌(㎎),페닐알라닌(㎎),트레오닌(㎎),트립토판(㎎),발린(㎎),히스티딘(㎎),아르기닌(㎎),비필수 아미노산(㎎),티로신(㎎),시스테인(㎎),알라닌(㎎),아스파르트산(㎎),글루탐산(㎎),글리신(㎎),프롤린(㎎),세린(㎎),타우린(㎎),글리신 베타인 (㎎),호마린(㎎),트리고넬린(㎎),리보핵산(㎎),데옥시리보핵산(㎎),콜레스테롤(g),콜레스테롤(㎎),총 지방산(g),총 필수 지방산(g),총 포화 지방산(g),총 포화 지방산(%),부티르산(4:0)(g),부티르산(4:0)(㎎),카프로산(6:0)(g),카프로산(6:0)(㎎),카프릴산(8:0)(g),카프릴산(8:0)(㎎),카프르산(10:0)(g),카프르산(10:0)(㎎),라우르산(12:0)(g),라우르산(12:0)(㎎),라우르산(12:0)(%),트라이데칸산(13:0)(㎎),미리스트산(14:0)(g),미리스트산(14:0)(㎎),미리스트산(14:0)(%),펜타데칸산(15:0)(㎎),팔미트산(16:0)(g),팔미트산(16:0)(㎎),팔미트산(16:0)(%),헵타데칸산(17:0)(㎎),스테아르산(18:0)(g),스테아르산(18:0)(㎎),스테아르산(18:0)(%),아라키드산(20:0)(g),아라키드산(20:0)(㎎),아라키드산(20:0)(%),헨에이코산산(21:0)(㎎),베헨산(22:0)(㎎),트리코산산(23:0)(㎎),리그노세르산(24:0)(㎎),총 단일 불포화지방산(g),총 단일 불포화지방산(%),미리스톨레산(14:1)(g),미리스톨레산(14:1)(㎎),미리스톨레산(14:1)(%),팔미톨레산(16:1)(g),팔미톨레산(16:1)(㎎),팔미톨레산(16:1)(%),헵타데센산(17:1)(㎎),올레산(18:1(n-9))(g),올레산(18:1(n-9))(㎎),올레산(18:1(n-9))(%),박센산(18:1(n-7))(g),박센산(18:1(n-7))(㎎),가돌레산(20:1)(g),가돌레산(20:1)(㎎),가돌레산(20:1)(%),에루크산(22:1)(㎎),에루크산(22:1)(%),네르본산(24:1)(㎎),총 다중 불포화지방산(g),총 다중 불포화지방산(%),리놀레산(18:2(n-6)c)(g),리놀레산(18:2(n-6)c)(㎎),리놀레산(18:2(n-6)c)(%),알파 리놀렌산(18:3(n-3))(g),알파 리놀렌산(18:3(n-3))(㎎),리놀렌산(18:3(n-3)(%),감마 리놀렌산(18:3(n-6))(g),감마 리놀렌산(18:3(n-6))(㎎),스테아리돈산(18:4)(%),에이코사디에노산(20:2(n-6))(g),에이코사디에노산(20:2(n-6))(㎎),에이코사트리에노산(20:3(n-3))(㎎),에이코사트리에노산(20:3(n-6))(g),에이코사트리에노산(20:3(n-6))(㎎),아라키돈산(20:4(n-6))(g),아라키돈산(20:4(n-6))(㎎),아라키돈산(20:4(n-6))(%),에이코사테트라에노산(20:4(n-3))(㎎),에이코사펜타에노산(20:5(n-3))(g),에이코사펜타에노산(20:5(n-3))(㎎),에이코사펜타에노산(20:5(n-3))(%),도코사디에노산(22:2)(㎎),도코사펜타에노산(22:5(n-3))(g),도코사펜타에노산(22:5(n-3))(㎎),도코사펜타에노산(22:5(n-3))(%),도코사헥사에노산(22:6(n-3))(g),도코사헥사에노산(22:6(n-3))(㎎),도코사헥사에노산(22:6(n-3))(%),EPA와 DHA의 합(㎎),오메가 3 지방산(g),오메가 6 지방산(g),트랜스 지방산(g),트랜스 올레산(18:1(n-9)t)(g),트랜스 올레산(18:1(n-9)t)(㎎),트랜스 리놀레산 (18:2t)(g),트랜스 리놀레산 (18:2t)(㎎),트랜스 리놀렌산(18:3t)(g),트랜스 리놀렌산(18:3t)(㎎),트랜스 리놀렌산(18:3t)(%),냉산가용성물질(㎎),총 불포화지방산(g),식염상당량(g),회분(g),가식부(%),산가용성물질(%),카페인(㎎)
0,1,음식,품목대표,꿩불고기,구이류,육류구이,500.0,g,-,-,368.8,-,412.6,-,33.500000,8.500000,0.0,39.700000,16.900000,7.200000,2.800000,2.800000,0.7,3.500000,0.0,0.0,0.0,9.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.105610,0.000004,0.000004,0.08539,0.458050,1.243120,1.243120,1.264310,0.00399,0.00032,0.0,0.00068,0.0,0.000048,0.0,0.0,0.0,0.0,0.0,0.000000,0.001425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00873,0.00006,0.0,0.0,0.0,0.0,0.000000,0.0,0.000330,0.0,0.003610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000063,0.0,0.0,0.000000,0.000000,0.00299,0.00299,0.0,0.0,0.0,0.0,0.0,1.284376,2.328599,2.484750,0.64519,1.274927,1.410841,0.0,1.367688,0.848539,2.205585,0.0,0.919806,0.276499,1.642381,2.781773,4.784606,1.419925,1.092295,1.264958,0.0,0.0,0.0,0.0,0.0,0.0,0.106180,0.106180,0.0,0.0,1.900000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.007115,0.0,0.0,0.0,0.031784,0.0,0.0,0.0,1.283841,0.0,0.0,0.0,0.527395,0.0,0.0,0.029012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.081352,0.0,0.0,0.0,2.307721,0.0,0.0,0.113463,0.0,0.027976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.236018,0.0,0.0,0.656723,0.0,0.0,0.018708,0.0,0.0,0.006649,0.0,0.0,0.000000,0.0,0.142840,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00

In [16]:
food_nutrients_copy3 = food_nutrients_copy2.copy()
delete_G_MG = integrated_G_MG[:]
delete_MG_UG = integrated_MG_UG[:]

In [17]:
for i in range(len(integrated_G_MG)):
    delete_G_MG[i] = integrated_G_MG[i] + '(㎎)'

for i in range(len(integrated_MG_UG)):
    delete_MG_UG[i] = integrated_MG_UG[i] + '(㎍)'

food_nutrients_copy3 = food_nutrients_copy2.drop(delete_G_MG, axis = 1)
food_nutrients_copy3 = food_nutrients_copy3.drop(delete_MG_UG, axis = 1)

['칼륨(㎎)', '비타민 C(㎎)', '콜린(㎎)', '총 아미노산(㎎)', '콜레스테롤(㎎)', '부티르산(4:0)(㎎)', '카프로산(6:0)(㎎)', '카프릴산(8:0)(㎎)', '카프르산(10:0)(㎎)', '라우르산(12:0)(㎎)', '미리스트산(14:0)(㎎)', '팔미트산(16:0)(㎎)', '스테아르산(18:0)(㎎)', '아라키드산(20:0)(㎎)', '미리스톨레산(14:1)(㎎)', '팔미톨레산(16:1)(㎎)', '올레산(18:1(n-9))(㎎)', '박센산(18:1(n-7))(㎎)', '가돌레산(20:1)(㎎)', '리놀레산(18:2(n-6)c)(㎎)', '알파 리놀렌산(18:3(n-3))(㎎)', '감마 리놀렌산(18:3(n-6))(㎎)', '에이코사디에노산(20:2(n-6))(㎎)', '에이코사트리에노산(20:3(n-6))(㎎)', '아라키돈산(20:4(n-6))(㎎)', '에이코사펜타에노산(20:5(n-3))(㎎)', '도코사펜타에노산(22:5(n-3))(㎎)', '도코사헥사에노산(22:6(n-3))(㎎)', '트랜스 올레산(18:1(n-9)t)(㎎)', '트랜스 리놀레산 (18:2t)(㎎)', '트랜스 리놀렌산(18:3t)(㎎)']


In [18]:
food_nutrients_copy3.info()
food_nutrients_copy2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58597 entries, 0 to 58596
Columns: 193 entries, NO to 카페인(㎎)
dtypes: float64(180), int64(1), object(12)
memory usage: 86.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58597 entries, 0 to 58596
Columns: 233 entries, NO to 카페인(㎎)
dtypes: float64(220), int64(1), object(12)
memory usage: 104.2+ MB


In [19]:
food_nutrients_copy3.loc[: ,'총내용량(g)':'총내용량(mL)'] =\
food_nutrients_copy3.loc[: ,'총내용량(g)':'총내용량(mL)'].replace(['-'], 0)

food_nutrients_copy3.loc[: ,'총내용량(g)':'총내용량(mL)'] = \
food_nutrients_copy3.loc[: ,'총내용량(g)':'총내용량(mL)'].astype(float)

In [20]:
print((food_nutrients_copy3['1회제공량'] == food_nutrients_copy3['총내용량(mL)']).sum())
print((food_nutrients_copy3['총내용량(mL)']>0.0).sum())
print((food_nutrients_copy3['1회제공량'] == food_nutrients_copy3['총내용량(g)']).sum())
print((food_nutrients_copy3['총내용량(g)']>0.0).sum())

(food_nutrients_copy3['1회제공량'] != food_nutrients_copy3['총내용량(mL)']).sum()

4475
7713
17899
35155


54122

In [21]:
# for col in food_nutrients_copy3.loc[: ,'단백질(g)':'카페인(㎎)']:
#     if col not in nutrient_persent_list:
#         food_nutrients_copy3[col] = food_nutrients_copy3[col] / food_nutrients_copy3['1회제공량'] *100

# food_nutrients_copy3.head()

'''데이터 통일성 부족;;;;;
어떤 식품은 1회제공량을 기준으로 영양소 량을 기록한게 있다면
어떤 식품은 총내용량을 기준으로 영양소 량을 기록한게 있다
1회 제공량 기준으로 비율을 구하면 값이 6000이상이 나오는 등 결치값 발생
각 식품별 영양소 비율을 구하는것은 포기.....
그냥 식품별 최대값을 구하고 그 최대값이 어떤 영양소인지만 구한다.
'''

'데이터 통일성 부족;;;;;\n어떤 식품은 1회제공량을 기준으로 영양소 량을 기록한게 있다면\n어떤 식품은 총내용량을 기준으로 영양소 량을 기록한게 있다\n1회 제공량 기준으로 비율을 구하면 값이 6000이상이 나오는 등 결치값 발생\n각 식품별 영양소 비율을 구하는것은 포기.....\n그냥 식품별 최대값을 구하고 그 최대값이 어떤 영양소인지만 구한다.\n'

In [22]:
fd = food_nutrients_copy3.loc[: ,'단백질(g)':'카페인(㎎)']
result = food_nutrients_copy3.loc[: ,'단백질(g)':'카페인(㎎)'].idxmax(axis=1)
food_nutrients_copy3['max_nutrient'] = result


max_value = food_nutrients_copy3.loc[: ,'단백질(g)':'카페인(㎎)'].max(axis=1)
food_nutrients_copy3['max_value'] = max_value

r = []
for i in range(len(result)):
    r.append(fd.iloc[i].drop(result[i]).max())

r

[33.5,
 44.9,
 31.6,
 11.561723041353,
 11.6,
 16.5,
 14.4,
 30.8,
 25.2,
 16.2,
 60.1,
 27.1,
 30.561400093193,
 29.4,
 16.2,
 26.9592466890526,
 26.9,
 43.0580921092491,
 31.7,
 23.9,
 42.4,
 5.4,
 20.0550757062532,
 32.1183333333333,
 8.3,
 10.0,
 6.1,
 2.8,
 10.1,
 35.7,
 16.7,
 7.8,
 8.4,
 33.0177773301579,
 6.7,
 13.0,
 11.9,
 19.2,
 4.27211688452746,
 11.6,
 6.8,
 12.0125394075668,
 20.3496853126553,
 14.4,
 23.4912924766319,
 34.4,
 52.0,
 14.7,
 45.7,
 29.2,
 11.4,
 2.85238675217711,
 1.57431954641941,
 1.8334,
 3.30617115754968,
 1.6497625,
 26.6404164891105,
 20.5500375051655,
 5.44890884927293,
 7.73826056743421,
 6.28945379609378,
 8.98514845148753,
 4.9247354751199,
 5.23704865192705,
 7.94975,
 8.63964223898382,
 5.46557210799641,
 10.7248380162142,
 3.642011601886,
 17.5421048944928,
 28.5729166666667,
 32.868725,
 37.8192222222222,
 16.9,
 15.5483111111111,
 12.9,
 42.2,
 17.6,
 20.0,
 47.9389722222222,
 33.6382083333333,
 39.62635,
 15.6,
 18.6,
 40.7166666666667,
 23

In [23]:
food_nutrients_copy3.head()

,NO,DB군,상용제품,식품명,식품대분류,식품상세분류,1회제공량,내용량_단위,총내용량(g),총내용량(mL),에너지(㎉),에너지(kj),수분(g),수분(%),단백질(g),지방(g),지질-가식부 100g당,탄수화물(g),총당류(g),자당(g),포도당(g),과당(g),유당(g),맥아당(g),갈락토오스(g),당알콜(g),에리스리톨(g),총 식이섬유(g),총 식이섬유(mg),총 식이섬유(%),수용성 식이섬유(g),불용성 식이섬유(g),셀룰로오스(%),리그닌(%),칼슘(㎎),철(㎎),마그네슘(㎎),인(㎎),칼륨(g),나트륨(㎎),아연(㎎),구리(㎎),망간(㎎),셀레늄(㎍),몰리브덴(㎍),요오드(㎍),염소(㎎),비타민 A(㎍),비타민 A(㎍ RE),레티놀(㎍),베타카로틴(㎍),레티놀A효능,비타민 D(D2+D3)(㎍),비타민 D2(㎍),비타민 D3(㎍),비타민 D1(㎍),비타민 E(㎎),비타민 E(㎎ α-TE),알파 토코페롤(㎎),베타 토코페롤(㎎),감마 토코페롤(㎎),델타 토코페롤(㎎),알파 토코트리에놀(㎎),베타 토코트리에놀(㎎),감마 토코트리에놀(㎎),델타 토코트리에놀(㎎),토코페롤(㎎),토코트리에놀(㎎),비타민 K(㎎),비타민 K1(㎍),비타민 K2(㎍),비타민 B1(㎎),비타민 B2(㎎),나이아신(㎎),나이아신(㎎ NE),나이아신(NE)(㎎),나이아신(NE)(㎎ NE),니코틴산 (㎎),니코틴아마이드(㎎),판토텐산(㎎),비타민 B6(㎎),피리독신(㎎),비오틴(㎍),엽산(DFE)(㎍),엽산 - 천연 엽산(㎍),엽산 - 합성 엽산(㎍),비타민 B12(㎎),비타민 C(g),콜린(g),총 아미노산(g),필수 아미노산(㎎),이소류신(㎎),류신(㎎),라이신(㎎),메티오닌(㎎),페닐알라닌(㎎),트레오닌(㎎),트립토판(㎎),발린(㎎),히스티딘(㎎),아르기닌(㎎),비필수 아미노산(㎎),티로신(㎎),시스테인(㎎),알라닌(㎎),아스파르트산(㎎),글루탐산(㎎),글리신(㎎),프롤린(㎎),세린(㎎),타우린(㎎),글리신 베타인 (㎎),호마린(㎎),트리고넬린(㎎),리보핵산(㎎),데옥시리보핵산(㎎),콜레스테롤(g),총 지방산(g),총 필수 지방산(g),총 포화 지방산(g),총 포화 지방산(%),부티르산(4:0)(g),카프로산(6:0)(g),카프릴산(8:0)(g),카프르산(10:0)(g),라우르산(12:0)(g),라우르산(12:0)(%),트라이데칸산(13:0)(㎎),미리스트산(14:0)(g),미리스트산(14:0)(%),펜타데칸산(15:0)(㎎),팔미트산(16:0)(g),팔미트산(16:0)(%),헵타데칸산(17:0)(㎎),스테아르산(18:0)(g),스테아르산(18:0)(%),아라키드산(20:0)(g),아라키드산(20:0)(%),헨에이코산산(21:0)(㎎),베헨산(22:0)(㎎),트리코산산(23:0)(㎎),리그노세르산(24:0)(㎎),총 단일 불포화지방산(g),총 단일 불포화지방산(%),미리스톨레산(14:1)(g),미리스톨레산(14:1)(%),팔미톨레산(16:1)(g),팔미톨레산(16:1)(%),헵타데센산(17:1)(㎎),올레산(18:1(n-9))(g),올레산(18:1(n-9))(%),박센산(18:1(n-7))(g),가돌레산(20:1)(g),가돌레산(20:1)(%),에루크산(22:1)(㎎),에루크산(22:1)(%),네르본산(24:1)(㎎),총 다중 불포화지방산(g),총 다중 불포화지방산(%),리놀레산(18:2(n-6)c)(g),리놀레산(18:2(n-6)c)(%),알파 리놀렌산(18:3(n-3))(g),리놀렌산(18:3(n-3)(%),감마 리놀렌산(18:3(n-6))(g),스테아리돈산(18:4)(%),에이코사디에노산(20:2(n-6))(g),에이코사트리에노산(20:3(n-3))(㎎),에이코사트리에노산(20:3(n-6))(g),아라키돈산(20:4(n-6))(g),아라키돈산(20:4(n-6))(%),에이코사테트라에노산(20:4(n-3))(㎎),에이코사펜타에노산(20:5(n-3))(g),에이코사펜타에노산(20:5(n-3))(%),도코사디에노산(22:2)(㎎),도코사펜타에노산(22:5(n-3))(g),도코사펜타에노산(22:5(n-3))(%),도코사헥사에노산(22:6(n-3))(g),도코사헥사에노산(22:6(n-3))(%),EPA와 DHA의 합(㎎),오메가 3 지방산(g),오메가 6 지방산(g),트랜스 지방산(g),트랜스 올레산(18:1(n-9)t)(g),트랜스 리놀레산 (18:2t)(g),트랜스 리놀렌산(18:3t)(g),트랜스 리놀렌산(18:3t)(%),냉산가용성물질(㎎),총 불포화지방산(g),식염상당량(g),회분(g),가식부(%),산가용성물질(%),카페인(㎎),max_nutrient,max_value
0,1,음식,품목대표,꿩불고기,구이류,육류구이,500.0,g,0.0,0.0,368.8,-,412.6,-,33.500000,8.500000,0.0,39.700000,16.900000,7.200000,2.800000,2.800000,0.7,3.500000,0.0,0.0,0.0,9.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.105610,0.000004,0.08539,0.458050,1.243120,1.264310,0.00399,0.00032,0.00068,0.000048,0.0,0.0,0.0,0.0,0.0,0.000000,0.001425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00873,0.00006,0.0,0.0,0.0,0.000000,0.000330,0.003610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000063,0.0,0.0,0.000000,0.00299,0.0,0.0,0.0,1.284376,2.328599,2.484750,0.64519,1.274927,1.410841,0.0,1.367688,0.848539,2.205585,0.0,0.919806,0.276499,1.642381,2.781773,4.784606,1.419925,1.092295,1.264958,0.0,0.0,0.0,0.0,0.0,0.0,0.106180,0.0,0.0,1.900000,0.0,0.0,0.0,0.0,0.000000,0.007115,0.0,0.0,0.031784,0.0,0.0,1.283841,0.0,0.0,0.527395,0.0,0.029012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.081352,0.0,0.0,2.307721,0.0,0.113463,0.027976,0.0,0.0,0.0,0.0,0.0,0.0,3.236018,0.0,0.656723,0.0,0.018708,0.0,0.006649,0.0,0.000000,0.142840,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.023427,0.0,0.0,0.0,0.0,0.100000,0.018116,0.053416,0.000000,0.0,0.0,0.0,0.0,5.800000,0.0,0.0,0.0,탄수화물(g),39.700000
1,2,음식,품목대표,닭갈비,구이류,육류구이,400.0,g,0.0,0.0,595.61,-,276.4,-,45.900000,25.800000,0.0,44.900000,21.200000,3.600000,5.900000,4.800000,0.0,6.900000,0.0,0.0,0.0,11.600000,0.0,0.0,0.0,0.0,0.0,0.0,0.098640,0.000003,0.10442,0.505250,1.200240,1.535830,0.00355,0.00034,0.00097,0.000058,0.0,0.0,0.0,0.0,0.0,0.000039,0.002133,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00682,0.00004,0.0,0.0,0.0,0.000240,0.000370,0.001230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000108,0.0,0.0,0.000001,0.00554,0.0,0.0,0.0,1.796242,3.245405,3.593312,0.87804,1.830141,2.019939,0.0,1.966315,1.198582,2.851723,0.0,

In [24]:
# food_nutrients_copy3.to_excel('통합 식품영양성분DB_정리본.xlsx')